# ESTUDO E RESOLUÇÃO DO CICLO DE RANKINE COM MODIFICAÇÕES

---

## Descrição do Problema Proposto

A figura a seguir contém o esquema do ciclo termodinâmico de potência
a ser avaliado neste trabalho:

![problema](./images/esquema-problema.png)

As hipóteses para serem adotadas para o ciclo:

 - Fluido de trabalho: vapor d’água;
 - Perda de carga desprezível nos equipamentos de troca térmica;
 - Operação isentrópica das turbinas e das bombas;
 - Vazão mássica (ṁ ) da corrente 1 fixa em 10.000 kg/h;
 - Saída do condensador (corrente 10): líquido saturado.

![problema](./images/tabela-correntes.png)

a) Determinar a eficiência térmica do sistema modificado proposto;

b) Obter a taxa de trabalho realizado em cada turbina; a taxa de calor
fornecido nas caldeiras; a taxa de trabalho consumida em cada bomba;

c) Determinar a vazão mássica de água de resfriamento empregada no
condensador.

d) Construir o diagrama T x s do sistema.

e) Realizar análise paramétrica no desempenho do ciclo considerando a
variação das pressões de descargas das turbinas; e a temperatura de saída
da caldeira 1; (para esta análise, plote gráficos para demonstrar a influência
desses parâmetros na eficiência térmica do ciclo). Discuta os resultados
obtidos.

In [147]:
# Pacotes necessários para a execução do código
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP

In [148]:
# Tabela onde serão armazenados os dados
colunas = ['Ponto', 'Pressão (kPa)', 'Temperatura (K)', 'Entalpia (kJ/kg)', 'Entropia (kJ/kg·K)']
df = pd.DataFrame(columns=colunas)

In [149]:
def obter_propriedades(P, T=None, x=None):
    """Função para obter propriedades termodinâmicas da água
    Args:
        P (float): Pressão em Pa
        T (float, optional): Temperatura em K. Defaults to None.
        x (float, optional): Título de vapor. Defaults to None.
    Returns:
        float, float: Entalpia específica e entropia específica
    """
    if T is not None:
        h = CP.PropsSI('H', 'P', P, 'T', T, 'Water')
        s = CP.PropsSI('S', 'P', P, 'T', T, 'Water')
    elif x is not None:
        h = CP.PropsSI('H', 'P', P, 'Q', x, 'Water')
        s = CP.PropsSI('S', 'P', P, 'Q', x, 'Water')
    else:
        raise ValueError("Especifique T ou x")
    return h, s

In [150]:
# Estado 1: Saída da Caldeira 1 (15 MPa, 600°C)

P1 = 15e6           # Pa
T1 = 600 + 273.15   # K
h1, s1 = obter_propriedades(P1, T=T1)

# Adicionando os dados à tabela
df.loc[len(df)] = ['1', P1/1000, T1, h1/1000, s1/1000]  # Convertendo para kJ/kg e kJ/kg·K
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.15,3583.131761,6.67959


In [151]:
# Ponto 2: Saída da Turbina 1 (4 MPa, expansão isentrópica)
P2 = 4e6
s2 = s1
try:
    h2 = CP.PropsSI('H', 'P', P2, 'S', s2, 'Water')
except:
    x2 = (s2 - CP.PropsSI('S', 'P', P2, 'Q', 0, 'Water')) / (CP.PropsSI('S', 'P', P2, 'Q', 1, 'Water') - CP.PropsSI('S', 'P', P2, 'Q', 0, 'Water'))
    h2 = CP.PropsSI('H', 'P', P2, 'Q', x2, 'Water')

# Adicionando o Ponto 2 à tabela
df.loc[len(df)] = ['2', P2/1000, CP.PropsSI('T', 'P', P2, 'H', h2, 'Water'), h2/1000, s2/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.67959
1,2,4000.0,647.806993,3153.874543,6.67959


In [152]:
# Ponto 3:
P3 = P2
s3 = s2
try:
    h3 = CP.PropsSI('H', 'P', P3, 'S', s3, 'Water')
except:
    x3 = (s3 - CP.PropsSI('S', 'P', P3, 'Q', 0, 'Water')) / (CP.PropsSI('S', 'P', P3, 'Q', 1, 'Water') - CP.PropsSI('S', 'P', P3, 'Q', 0, 'Water'))
    h3 = CP.PropsSI('H', 'P', P3, 'Q', x3, 'Water')

# Adicionando o Ponto 3 à tabela
df.loc[len(df)] = ['3', P3/1000, CP.PropsSI('T', 'P', P3, 'H', h3, 'Water'), h3/1000, s3/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.67959
1,2,4000.0,647.806993,3153.874543,6.67959
2,3,4000.0,647.806993,3153.874543,6.67959


In [153]:
# Ponto 4: (igual ao Ponto 3)
df.loc[len(df)] = ['4', P3/1000, CP.PropsSI('T', 'P', P3, 'H', h3, 'Water'), h3/1000, s3/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.67959
1,2,4000.0,647.806993,3153.874543,6.67959
2,3,4000.0,647.806993,3153.874543,6.67959
3,4,4000.0,647.806993,3153.874543,6.67959


In [154]:
# Ponto 5: Saída da Caldeira 2 (4 MPa, 600°C) - Correção da errata
P5 = 4e6
T5 = 600 + 273.15
h5, s5 = obter_propriedades(P5, T=T5)

# Adicionando o Ponto 5 à tabela
df.loc[len(df)] = ['5', P5/1000, T5, h5/1000, s5/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.679590
1,2,4000.0,647.806993,3153.874543,6.679590
2,3,4000.0,647.806993,3153.874543,6.679590
3,4,4000.0,647.806993,3153.874543,6.679590
4,5,4000.0,873.150000,3674.867523,7.370528


In [155]:
# Ponto 6: Saída da Turbina 2 (500 kPa, expansão isentrópica)
P6 = 5e5
s6 = s5
try:
    h6 = CP.PropsSI('H', 'P', P6, 'S', s6, 'Water')
except:
    x6 = (s6 - CP.PropsSI('S', 'P', P6, 'Q', 0, 'Water')) / (CP.PropsSI('S', 'P', P6, 'Q', 1, 'Water') - CP.PropsSI('S', 'P', P6, 'Q', 0, 'Water'))
    h6 = CP.PropsSI('H', 'P', P6, 'Q', x6, 'Water')

# Adicionando o Ponto 6 à tabela
df.loc[len(df)] = ['6', P6/1000, CP.PropsSI('T', 'P', P6, 'H', h6, 'Water'), h6/1000, s6/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.679590
1,2,4000.0,647.806993,3153.874543,6.679590
2,3,4000.0,647.806993,3153.874543,6.679590
3,4,4000.0,647.806993,3153.874543,6.679590
4,5,4000.0,873.150000,3674.867523,7.370528
5,6,500.0,548.500710,3013.589893,7.370528


In [156]:
# Ponto 7: (igual ao Ponto 6)
df.loc[len(df)] = ['7', P6/1000, CP.PropsSI('T', 'P', P6, 'H', h6, 'Water'), h6/1000, s6/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.679590
1,2,4000.0,647.806993,3153.874543,6.679590
2,3,4000.0,647.806993,3153.874543,6.679590
3,4,4000.0,647.806993,3153.874543,6.679590
4,5,4000.0,873.150000,3674.867523,7.370528
5,6,500.0,548.500710,3013.589893,7.370528
6,7,500.0,548.500710,3013.589893,7.370528


In [157]:
# Ponto 8: (igual ao Ponto 6)
df.loc[len(df)] = ['8', P6/1000, CP.PropsSI('T', 'P', P6, 'H', h6, 'Water'), h6/1000, s6/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.679590
1,2,4000.0,647.806993,3153.874543,6.679590
2,3,4000.0,647.806993,3153.874543,6.679590
3,4,4000.0,647.806993,3153.874543,6.679590
4,5,4000.0,873.150000,3674.867523,7.370528
5,6,500.0,548.500710,3013.589893,7.370528
6,7,500.0,548.500710,3013.589893,7.370528
7,8,500.0,548.500710,3013.589893,7.370528


In [158]:
# Ponto 9: Saída da Turbina 2
P9 = 10e3
s9 = s6

try:
    h9 = CP.PropsSI('H', 'P', P9, 'S', s9, 'Water')
except:
    x9 = (s9 - CP.PropsSI('S', 'P', P9, 'Q', 0, 'Water')) / (CP.PropsSI('S', 'P', P9, 'Q', 1, 'Water') - CP.PropsSI('S', 'P', P9, 'Q', 0, 'Water'))
    h9 = CP.PropsSI('H', 'P', P9, 'Q', x9, 'Water')

# Adicionando o Ponto 9 à tabela
df.loc[len(df)] = ['9', P9/1000, CP.PropsSI('T', 'P', P9, 'H', h9, 'Water'), h9/1000, s9/1000]
df

,Ponto,Pressão (kPa),Temperatura (K),Entalpia (kJ/kg),Entropia (kJ/kg·K)
0,1,15000.0,873.150000,3583.131761,6.679590
1,2,4000.0,647.806993,3153.874543,6.679590
2,3,4000.0,647.806993,3153.874543,6.679590
3,4,4000.0,647.806993,3153.874543,6.679590
4,5,4000.0,873.150000,3674.867523,7.370528
5,6,500.0,548.500710,3013.589893,7.370528
6,7,500.0,548.500710,3013.589893,7.370528
7,8,500.0,548.500710,3013.589893,7.370528
8,9,10.0,318.956329,2335.617568,7.370528


In [159]:
# Ponto 10: Saída do Condensador
P10 = P9